In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [2]:
unp = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Unpaid_Services_sample.csv")
unp.printSchema()
unp.registerTempTable('u')
unp.show()

root
 |-- Keydate_prompt: integer (nullable = true)
 |-- IS_Locked: integer (nullable = true)
 |-- OIB: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Due_Days: integer (nullable = true)
 |-- Document_Type: string (nullable = true)
 |-- Document_Type_Subgroup: string (nullable = true)
 |-- Document_Type_Description: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)

+--------------+---------+-----------+-------+--------+-------------+----------------------+-------------------------+----+----+----+
|Keydate_prompt|IS_Locked|        OIB| Amount|Due_Days|Document_Type|Document_Type_Subgroup|Document_Type_Description| _c8| _c9|_c10|
+--------------+---------+-----------+-------+--------+-------------+----------------------+-------------------------+----+----+----+
|      20170731|        1|80188553401|2310.25|     530|           CF|        COURT DOCUMENT|            Sudski trošak|null|null

In [5]:
unp_df = spark.sql("select * from u where Document_Type not in ('CF','EN','IC','NP','PO','SR')").show(2)

+--------------+---------+-----------+------+--------+-------------+----------------------+-------------------------+----+----+----+
|Keydate_prompt|IS_Locked|        OIB|Amount|Due_Days|Document_Type|Document_Type_Subgroup|Document_Type_Description| _c8| _c9|_c10|
+--------------+---------+-----------+------+--------+-------------+----------------------+-------------------------+----+----+----+
|      20170731|        1|30170698412|654.42|    4454|           MZ|            RECEIVABLE|            Mig. troškovi|null|null|null|
|      20170731|        1|48435541296|254.81|    4392|           MZ|            RECEIVABLE|            Mig. troškovi|null|null|null|
+--------------+---------+-----------+------+--------+-------------+----------------------+-------------------------+----+----+----+
only showing top 2 rows



In [4]:
lock=spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Lock_by_DocNum_-_Complaints.csv")
lock.printSchema()

df=unp_df.join(lock,"document_number","inner")
print(df)
df.registerTempTable('l')

root
 |-- Document_Number: long (nullable = true)
 |-- Lock_From_Date: integer (nullable = true)
 |-- Lock_To_Date: integer (nullable = true)



AnalysisException: 'USING column `document_number` cannot be resolved on the left side of the join. The left-side columns: [Keydate_prompt, IS_Locked, OIB, Amount, Due_Days, Document_Type, Document_Type_Subgroup, Document_Type_Description, _c8, _c9, _c10];'

In [ ]:
df1=spark.sql("SELECT * FROM l WHERE l.Keydate not between l.Lock_From_Date AND l.Lock_To_Date").show()

In [ ]:
df2=df1.withColumn("Interval",f.expr("case when Due_Days<=0 then 'ON_TIME'\
                                        when Due_Days>=44 and Due_Days<=63 then 4463\
                                        when Due_Days>=64 and Due_Days<=93 then 6493\
                                        when Due_Days>=94  then 94\
                                        when Due_Days>=1 and Due_Days<=43 then 'remainder' end"))

df2.show()
df2.registerTempTable('v')

In [ ]:
df3=spark.sql("select v.Keydate_prompt,v.oib,v.amount,v.Interval from v group by v.Keydate_prompt,v.oib,v.amount,v.Interval")
                                    
df3.show()